# Exploring Debugging Book Slicer

Andreas Zeller: "Tracking Failure Origins". In Andreas Zeller, "The Debugging Book", https://www.debuggingbook.org/html/Slicer.html. 

In [1]:
from debuggingbook.Slicer import Slicer
from debuggingbook.Slicer import *
from debuggingbook.bookutils import print_content

Define a few test functions. 

In [2]:
def test1(param):
    n = param
    i = 1
    sum = 0
    prod = 1 
    while i <= n:
        sum = sum + i
        prod = prod * i
        i = i + 1
    print(sum)
    print(prod)

In [3]:
def test2(param):
    n = param 
    arr = [-4, 3, 2]
    i = 1
    z = 0
    while i <= n:
        x = arr[i-1]
        if x < 0:
            y = x + 1
        else:
            y = x + 2
        z = y
        i = i + 1
    print(z)

In [4]:
def test3(param):
    n = param
    if n > 1:
        n = 11
    else:
        n = 12
    print(n)

In [5]:
with Slicer(log=True) as slicer:
    test1(2)

Instrumenting <function test1 at 0x7fb898912b80>
def test1(param):
    _data.param('param', param, pos=1, last=True)
    n = _data.set('n', _data.get('param', param))
    i = _data.set('i', 1)
    sum = _data.set('sum', 0)
    prod = _data.set('prod', 1)
    while _data.test(_data.get('i', i) <= _data.get('n', n)):
        with _data:
            sum = _data.set('sum', _data.get('sum', sum) + _data.get('i', i))
            prod = _data.set('prod', _data.get('prod', prod) * _data.get('i', i))
            i = _data.set('i', _data.get('i', i) + 1)
    _data.ret(_data.call(_data.get('print', print))(_data.arg(_data.get('sum', sum), pos=1)))
    _data.ret(_data.call(_data.get('print', print))(_data.arg(_data.get('prod', prod), pos=1)))

3
2


In [6]:
slicer.code()

*    1 def test1(param):
*    2     n = param  # <= param (1)
*    3     i = 1
*    4     sum = 0
*    5     prod = 1
*    6     while i <= n:  # <= i (9), i (3), n (2)
*    7         sum = sum + i  # <= sum (7), sum (4), i (9), i (3); <- <test> (6)
*    8         prod = prod * i  # <= i (9), prod (5), i (3), prod (8); <- <test> (6)
*    9         i = i + 1  # <= i (9), i (3); <- <test> (6)
    10     print(sum)
    11     print(prod)


In [7]:
print_content(inspect.getsource(test1), '.py', start_line_number=1)

 1  def test1(param):
 2      n = param
 3      i = 1
 4      sum = 0
 5      prod = 1 
 6      while i <= n:
 7          sum = sum + i
 8          prod = prod * i
 9          i = i + 1
10      print(sum)
11      print(prod)

In [8]:
test1_tree = ast.parse(inspect.getsource(test1))

with DataTrackerTester(test1_tree, test1):
    test1(1)


1
1


In [9]:
with DependencyTrackerTester(test1_tree, test1) as call_deps:
    test1(2)
    
call_deps.dependencies()

3
2
